In [1]:
from selenium import webdriver
from tqdm import tqdm
import pandas as pd
from csv import reader
from requests_html import AsyncHTMLSession

In [ ]:
##rangos de valores 0 -150,000,000 :: +20,000,000

In [9]:
url = 'https://listado.mercadolibre.com.co/inmuebles/venta/antioquia/inmuebles-en-venta-antioquia_NoIndex_True#unapplied_filter_id%3Dprice%26unapplied_filter_name%3DPrecio%26unapplied_value_id%3D170000000-190000000%26unapplied_value_name%3D%24170.000.000+a+%24190.000.000%26unapplied_autoselect%3Dfalse'
driver = webdriver.Chrome(executable_path = '/Users/Thony/Documents/chromedriver')
driver.get(url)
driver.find_element_by_css_selector('#newCookieDisclaimerButton').click()
driver.find_element_by_css_selector('body > div.onboarding-cp > div > div > div.andes-tooltip-inner > div > button').click()

In [10]:
driver.find_element_by_css_selector('#root-app > div > div > aside > section.ui-search-filter-groups > div:nth-child(5) > ul > li:nth-child(4) > form > div:nth-child(1) > label > div.andes-form-control__control > input').send_keys('0')
driver.find_element_by_css_selector('#root-app > div > div > aside > section.ui-search-filter-groups > div:nth-child(5) > ul > li:nth-child(4) > form > div:nth-child(2) > label > div.andes-form-control__control > input').send_keys('150000000')
driver.find_element_by_css_selector('#root-app > div > div > aside > section.ui-search-filter-groups > div:nth-child(5) > ul > li:nth-child(4) > form > div:nth-child(3) > button').click()

In [11]:
links = []

In [200]:
condition = True
while condition:
    bloques = driver.find_elements_by_css_selector('#root-app > div > div > section > ol > *')
    for r in bloques:
        r1 = r.find_elements_by_css_selector(' ol > * > div > div')
        r2 = r.find_element_by_tag_name('a')
        links.append(r2.get_property('href'))
        print('Número de link extraído: ',len(links))
    try:
        driver.find_element_by_css_selector('#root-app > div > div > section > div.ui-search-pagination > ul > li.andes-pagination__button.andes-pagination__button--next > a').click()
    except:
        condition = False

Número de link extraído:  36489
Número de link extraído:  36490
Número de link extraído:  36491
Número de link extraído:  36492
Número de link extraído:  36493
Número de link extraído:  36494
Número de link extraído:  36495
Número de link extraído:  36496
Número de link extraído:  36497
Número de link extraído:  36498
Número de link extraído:  36499
Número de link extraído:  36500
Número de link extraído:  36501
Número de link extraído:  36502
Número de link extraído:  36503
Número de link extraído:  36504
Número de link extraído:  36505
Número de link extraído:  36506
Número de link extraído:  36507
Número de link extraído:  36508
Número de link extraído:  36509
Número de link extraído:  36510
Número de link extraído:  36511
Número de link extraído:  36512
Número de link extraído:  36513
Número de link extraído:  36514
Número de link extraído:  36515
Número de link extraído:  36516
Número de link extraído:  36517
Número de link extraído:  36518
Número de link extraído:  36519
Número d

In [198]:
driver.find_element_by_css_selector('#root-app > div > div > aside > section.ui-search-applied-filters > a:nth-child(3) > div > button').click()

In [199]:
driver.find_element_by_css_selector('#root-app > div > div > aside > section.ui-search-filter-groups > div:nth-child(5) > ul > li:nth-child(4) > form > div:nth-child(1) > label > div.andes-form-control__control > input').send_keys('2100000000')
driver.find_element_by_css_selector('#root-app > div > div > aside > section.ui-search-filter-groups > div:nth-child(5) > ul > li:nth-child(4) > form > div:nth-child(2) > label > div.andes-form-control__control > input').send_keys('100000000000')
driver.find_element_by_css_selector('#root-app > div > div > aside > section.ui-search-filter-groups > div:nth-child(5) > ul > li:nth-child(4) > form > div:nth-child(3) > button').click()

In [201]:
len(links)
link_1 = pd.unique(links)
len(link_1)

38168

In [202]:
linkss = pd.DataFrame(link_1)
linkss.columns = ['link_mercado_libre']
linkss.head()

,link_mercado_libre
0,https://casa.mercadolibre.com.co/MCO-855594234...
1,https://apartamento.mercadolibre.com.co/MCO-85...
2,https://casa.mercadolibre.com.co/MCO-854844561...
3,https://apartamento.mercadolibre.com.co/MCO-85...
4,https://lote-casalote.mercadolibre.com.co/MCO-...


In [203]:
linkss.to_csv("mercado_libre_links_antioquia20220207.csv",index = True)

In [20]:
""" urls = []
with open('links_mercado_libre_cordoba.csv','r') as f:
    csv_reader = reader(f)
    for row in csv_reader:
        urls.append(row[1]) """

In [212]:
urls = links
print(len(urls))

38169


In [213]:
all_in = []
try:
    for i in tqdm(urls):
        s = AsyncHTMLSession()
        r = await s.get(i)
        try:
            title = r.html.find('div > div.ui-pdp-header__title-container > h1',first = True).text
            price = r.html.find('span.andes-money-amount__fraction',first = True).text
        except:
            title = "None"
            price = "None"
        try:
            tipo_r = r.html.find('div > div.ui-pdp-header__subtitle > span',first = True).text
        except:
            tipo_r = 'None'
        try:
            tiempo_anuncio = r.html.find('div.ui-pdp-container__row.ui-pdp-container__row--header > div > p',first = True).text
        except:
            tiempo_anuncio = 'None'
        try:
            nombre = r.html.find('#seller_profile > div > div > div > h3',first = True).text
            lugar = r.html.find('div.ui-pdp-media.ui-vip-location__subtitle.ui-pdp-color--BLACK > div > p',first = True).text    
            descripcion = r.html.find('div.ui-pdp-container__row.ui-pdp-container__row--description > div > p',first = True).text.strip('\n')
            props = []
            prop = r.html.find('div.ui-pdp-highlighted-specs-res div span')
            for p in prop:
                props.append(p.text)
        except:
            nombre = 'None'
            lugar = 'None'
            descripcion = 'None'
            props = "None"
            
        try:
            id_t = r.html.find('p.ui-seller-info__status-info__subtitle',first = True).text
        except:
            id_t = 'None'
            
        try:
            caracts = []
            caract = r.html.find('div.ui-pdp-specs__table table tbody tr th')
            for c in caract:
                caracts.append(c.text)
                valores = []
                values = r.html.find('div.ui-pdp-specs__table table tbody tr td')
            for val in values:
                valores.append(val.text)
            caracteristicas = dict(zip(caracts,valores))
        except:
            caracteristicas = 'None'
        try:
            ambs = []
            amb = r.html.find('div.andes-tab-content')
            for a in amb:
                ambs.append(a.text)
        except:
            ambs = 'None'

        info = 'script:contains("window.__PRELOADED_STATE__")'
        script_txt = r.html.find(info, first = True)
        latitude = script_txt.search('"latitude":"{latitude}"')
        longitude = script_txt.search('"longitude":"{longitude}"')
        dict_reg = {
            'titulo':title,
            'tipo_registo': tipo_r,
            'tiempo_anuncio':tiempo_anuncio,
            'Nombre_anunciante': nombre,
            'Ubicacion': lugar,
            'descripcion': descripcion,
            'propiedad':props,
            'id':id_t,
            'caracteristicas':caracteristicas,
            'ambiente':ambs,
            'url':i,
            'longitude':longitude,
            'latitude':latitude,
            'precio':price
        }
        all_in.append(dict_reg)
except:
        all_in.append({})

100%|██████████| 38169/38169 [8:47:18<00:00,  1.21it/s]   


In [214]:
df = pd.DataFrame(all_in)

In [ ]:
df.

In [222]:
df['Nombre_anunciante'] = df['Nombre_anunciante'].str.replace('\n','')
df['titulo'] = df['titulo'].str.replace('\n','')
df['descripcion'] = df['descripcion'].str.replace('\n','')
df['Nombre_anunciante'] = df['Nombre_anunciante'].str.replace('\\n','')
df['titulo'] = df['titulo'].str.replace('\\n','')
df['descripcion'] = df['descripcion'].str.replace('\\n','')
df['Nombre_anunciante'] = df['Nombre_anunciante'].str.replace('\r','')
df['titulo'] = df['titulo'].str.replace('\r','')
df['descripcion'] = df['descripcion'].str.replace('\r','')
df['Nombre_anunciante'] = df['Nombre_anunciante'].str.replace('\\','')
df['titulo'] = df['titulo'].str.replace('\\','')
df['descripcion'] = df['descripcion'].str.replace('\\','')
df['Nombre_anunciante'] = df['Nombre_anunciante'].str.replace(';','')
df['titulo'] = df['titulo'].str.replace(';','')
df['descripcion'] = df['descripcion'].str.replace(';','')

In [221]:
df.head()

,titulo,tipo_registo,tiempo_anuncio,Nombre_anunciante,Ubicacion,descripcion,propiedad,id,caracteristicas,ambiente,url,longitude,latitude,precio
0,Casa Prefabricada Con Diseño Personalizado Y P...,Casa en Venta,Publicado hace 3 días,Juan Carlos,"Girardota, Antioquia, Colombia, Girardota, Ant...","Casas prefabricadas tipo cubo y campestre, ela...","[121 m² totales, 5 habitaciones, 3 baños]",None,"{'Área total': '121 m²', 'Área construida': '1...","[Baño de visitas\nComedor\nBalcón\nCocina, Par...",https://casa.mercadolibre.com.co/MCO-855594234...,<Result () {'longitude': '-75.4455662'}>,<Result () {'latitude': '6.3774241'}>,55.000.000
1,"Apartamento En Venta En Calatrava, Itagui.",Apartamento en Venta,Publicado hace 6 días,Juanc Pavas,"Calatrava, Itaguí, Antioquia",CÓDIGO DEL INMUEBLE: 3280329Apartamento en ven...,[55 m² totales],5 años,"{'Área total': '55 m²', 'Área construida': '55...",[Estaciones de metro\nEnvigado\n24 mins - 1.88...,https://apartamento.mercadolibre.com.co/MCO-85...,<Result () {'longitude': '-75.6088719'}>,<Result () {'latitude': '6.1804631'}>,130.000.000
2,"Casa En Venta En Guayabal, Medellin.",Casa en Venta,Publicado hace 6 días,Juanc Pavas,"Guayabal, Medellín, Antioquia",CODIGO DEL INMUEBLE: 1181223Casa en venta en G...,"[100 m² totales, 3 habitaciones, 1 baño]",5 años,"{'Área total': '100 m²', 'Área construida': '1...",[Estaciones de metro\nPoblado\n16 mins - 1.264...,https://casa.mercadolibre.com.co/MCO-854844561...,<Result () {'longitude': '-75.5830831'}>,<Result () {'latitude': '6.2192788'}>,150.000.000
3,Apartaestudio En Venta En Bello Agua Clara,Apartamento en Venta,Publicado hace 4 días,Inmobiliariavivenorte,"Bello, Antioquia","Agua Clara:1 habitaciones, 1 baño, salón comed...","[32 m² totales, 1 habitación, 1 baño]",2 años,"{'Área total': '32 m²', 'Área construida': '32...",[Colegios\nInstitución Educativa Villa Baena\n...,https://apartamento.mercadolibre.com.co/MCO-85...,<Result () {'longitude': '-75.575'}>,<Result () {'latitude': '6.343'}>,127.000.000
4,Santa Elena,Lote y casalote en Venta,Publicado hace 3 días,Urban In Motion,"Santa Elena Sn, Santa Elena, Medellín, Antioquia",Predios a 10 minutos de la Iglesia de Barro Bl...,[899 ha totales],1 año,"{'Área total': '899 ha', 'Acceso': 'Asfalto'}",[],https://lote-casalote.mercadolibre.com.co/MCO-...,<Result () {'longitude': '-75.4982477'}>,<Result () {'latitude': '6.2101762'}>,78.000.000


In [223]:
df.to_csv('mercadolibre_antioquia_data_todos_20220208.csv',index = False,sep =';')